In [94]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
df = pd.read_csv('sap_storing_data_hu_project.csv', low_memory = False)  

# Remove duplicates, date time convert, hersteltijd created, dropped nans

In [3]:
df = df.drop_duplicates(subset=['#stm_sap_meldnr'])
df = df.reset_index(drop=True)

df['stm_aanntpl_tijd']= pd.to_datetime(df['stm_aanntpl_tijd'])
df['stm_fh_ddt']= pd.to_datetime(df['stm_fh_ddt'])

# Calculate the true reparation time
df['stm_hers_tijd'] = (df['stm_fh_ddt'].dt.hour * 60 + df['stm_fh_ddt'].dt.minute) - (df['stm_aanntpl_tijd'].dt.hour * 60 + df['stm_aanntpl_tijd'].dt.minute)

df = df.dropna(subset=['stm_hers_tijd','stm_prioriteit' ]) 
df = df.reset_index(drop=True)

# Linear regression prioriteit

In [4]:
# df_og = df[['#stm_sap_meldnr', 'stm_fh_duur', 'stm_oorz_groep','stm_prioriteit', 'stm_geo_mld']]

# df_og = df_og.drop_duplicates(subset=['#stm_sap_meldnr'])

# df_og = df_og.reset_index(drop=True)

# df_og = df_og.dropna(subset=['stm_fh_duur','stm_prioriteit' ]) 

# df_og = df_og.reset_index(drop=True)

# for c, i in enumerate(df_og['stm_geo_mld']):
#     try:
#         df_og.at[c, 'stm_geo_mld'] = int(float(i))
#     except:
#         df_og.drop([c], inplace=True)
        
# df_og = df_og.reset_index(drop=True)

In [5]:
# # decide what the quantiles are
# Q1 = df_og['stm_fh_duur'].quantile(0.25)
# Q3 = df_og['stm_fh_duur'].quantile(0.75)
# IQR = Q3 - Q1
# print(IQR)

# # delete the outliers
# only_duur = df_og['stm_fh_duur']
# df_og['strong_outlier'] = ((only_duur < (Q1 - 1.5 * IQR)) | (only_duur > (Q3 + 1.5 * IQR))) #create temporary column
# df_og = df_og.drop(df_og[df_og.strong_outlier == True].index)

# df_og =  df_og.drop('strong_outlier', 1)


In [6]:
# # DummiesX = pd.get_dummies(data=df_og[['stm_geo_mld']])#, 'stm_prioriteit', 'stm_oorz_groep']])
# # DummiesX = pd.concat([DummiesX,df_og['stm_prioriteit']], axis = 1)
# # DummiesX.head()
# X = df_og[['stm_prioriteit']]
# y = df_og['stm_fh_duur']

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [8]:
# linreg = LinearRegression()
# linreg.fit(X_train, y_train)

# # print(linreg.intercept_)
# # print(linreg.coef_)

# y_pred = linreg.predict(X_test)

# np.sqrt(metrics.mean_squared_error(y_test,y_pred))
# r2_score(y_test,y_pred)

In [9]:
# y_train.mean()

In [10]:
# np.sqrt(metrics.mean_squared_error(y_test,y_pred))

# Decision Tree regression prioriteit en oorzaak groep

In [11]:
from sklearn.tree import DecisionTreeRegressor

In [12]:
df_pog = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_geo_mld']]

## Drop duplicates, drop nan's

In [13]:
# Drop alle duplicates mbv #stm_sap_meldnr
df_pog = df_pog.drop_duplicates(subset=['#stm_sap_meldnr'])

df_pog = df_pog.reset_index(drop=True)

# Drop rij als er maar 1 nan values tussen deze kolommen zit.
df_pog = df_pog.dropna(subset=['stm_hers_tijd','stm_prioriteit', 'stm_oorz_groep']) 

df_pog = df_pog.reset_index(drop=True)

## Drop outliers

In [14]:
df_pog = df_pog[(df_pog['stm_hers_tijd'] >= 4) & (df_pog['stm_hers_tijd'] <= 480)]

df_pog = df_pog.reset_index(drop=True)

## Get dummies for oorzaakgroep and concat with prioriteit

In [15]:
DummiesX = pd.get_dummies(data=df_pog[['stm_oorz_groep']])#, 'stm_prioriteit', 'stm_oorz_groep']])
DummiesX = pd.concat([DummiesX,df_pog['stm_prioriteit']], axis = 1)
DummiesX.head()
y = df_pog['stm_hers_tijd']

## Train data

In [16]:
X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)

In [17]:
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_2 = DecisionTreeRegressor(max_depth=7)
regr_1.fit(X_train,y_train)
regr_2.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=7)

## Predict data

In [18]:
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

In [19]:
np.sqrt(metrics.mean_squared_error(y_test,y_1))

76.57518088078423

In [20]:
np.sqrt(metrics.mean_squared_error(y_test,y_2))

76.54826025960315

In [21]:
r2_score(y_test, y_1)

0.026215099353142812

In [22]:
r2_score(y_test, y_2)

0.026899662839435123

# Decision Tree regression prioriteit en oorzaak groep en oorzaakcode

In [23]:
df_Dpogc = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']]

In [24]:
# Drop alle duplicates mbv #stm_sap_meldnr
df_Dpogc = df_Dpogc.drop_duplicates(subset=['#stm_sap_meldnr'])

df_Dpogc = df_Dpogc.reset_index(drop=True)

# Drop rij als er maar 1 nan values tussen deze kolommen zit.
df_Dpogc = df_Dpogc.dropna(subset=['stm_hers_tijd','stm_prioriteit', 'stm_oorz_groep', 'stm_oorz_code']) 

df_Dpogc = df_Dpogc.reset_index(drop=True)

In [25]:
df_Dpogc

,#stm_sap_meldnr,stm_hers_tijd,stm_oorz_groep,stm_prioriteit,stm_oorz_code
0,50053211,540.0,ONR-RIB,9.0,133.0
1,50053213,806.0,ONR-DERD,9.0,143.0
2,50053214,1040.0,ONR-DERD,9.0,142.0
3,50053215,1356.0,ONR-DERD,9.0,142.0
4,50053218,690.0,TECHONV,9.0,218.0
...,...,...,...,...,...
445899,99003503,179.0,TECHONV,4.0,221.0
445900,99003504,3.0,TECHONV,5.0,298.0
445901,99003505,24.0,TECHONV,5.0,215.0
445902,99003506,36.0,TECHONV,2.0,218.0


In [26]:
df_Dpogc = df_Dpogc[(df_Dpogc['stm_hers_tijd'] >= 4) & (df_Dpogc['stm_hers_tijd'] <= 480)]
# TODO: waarom zoveel weggegooid
df_Dpogc = df_Dpogc.reset_index(drop=True)

In [27]:
df_Dpogc['stm_hers_tijd'].mean()

67.70978782451179

In [28]:
df_Dpogc.describe()

,#stm_sap_meldnr,stm_hers_tijd,stm_prioriteit,stm_oorz_code
count,3.444790e+05,344479.000000,344479.000000,344479.000000
mean,7.987887e+07,67.709788,3.715713,200.765814
std,3.414743e+06,77.937001,1.380167,44.707157
min,5.005326e+07,4.000000,1.000000,33.000000
25%,8.045409e+07,19.000000,2.000000,151.000000
50%,8.059458e+07,40.000000,4.000000,215.000000
75%,8.073855e+07,83.000000,5.000000,221.000000
max,1.000000e+08,480.000000,9.000000,999.000000


In [29]:
DummiesX = pd.get_dummies(data=df_Dpogc[['stm_oorz_groep', 'stm_oorz_code']])
DummiesX = pd.concat([DummiesX,df_Dpogc['stm_prioriteit']], axis = 1)
DummiesX.head()
y = df_Dpogc['stm_hers_tijd']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)

In [31]:
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_2 = DecisionTreeRegressor(max_depth=7)
regr_1.fit(X_train,y_train)
regr_2.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=7)

In [32]:
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

In [33]:
np.sqrt(metrics.mean_squared_error(y_test,y_1))

75.91898445279033

In [34]:
np.sqrt(metrics.mean_squared_error(y_test,y_2))

75.0833719924327

In [35]:
r2_score(y_test, y_1)

0.042832919818925896

In [36]:
r2_score(y_test, y_2)

0.06378733794207447

# Random Forest Regression

In [37]:
from sklearn.ensemble import RandomForestRegressor

## RFR oorzaakcode, oorzaak groep en prioriteit

In [38]:
df_Kpog = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']]

In [39]:
df_Kpog = df_Kpog.drop_duplicates(subset=['#stm_sap_meldnr'])

df_Kpog = df_Kpog.reset_index(drop=True)

df_Kpog = df_Kpog.dropna(subset=['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit', 'stm_oorz_groep']) # 

df_Kpog = df_Kpog.reset_index(drop=True)

In [40]:
# decide what the quantiles are
Q1 = df_Kpog['stm_hers_tijd'].quantile(0.25)
Q3 = df_Kpog['stm_hers_tijd'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

df_Kpog = df_Kpog[(df_Kpog['stm_hers_tijd'] >= 4) & (df_Kpog['stm_hers_tijd'] <= 480)]

df_Kpog = df_Kpog.reset_index(drop=True)

73.0


In [41]:
DummiesX = pd.get_dummies(data=df_Kpog[['stm_oorz_code', 'stm_oorz_groep']])#, 'stm_prioriteit', 'stm_oorz_groep']])
DummiesX = pd.concat([DummiesX,df_Kpog['stm_prioriteit']], axis = 1)
DummiesX.head()
# DummiesX = df_Kpog[['stm_prioriteit']]
y = df_Kpog['stm_hers_tijd']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)

In [43]:
# def best_max_depth_calculator():
#     depthlist = []
#     depth_r2 = []
#     for depth in range(1,15):
#         regr = RandomForestRegressor(max_depth=depth, random_state=0)
#         regr.fit(DummiesX, y)
#         y_pred = regr.predict(X_test)
#         depthlist.append(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
#         depth_r2.append(r2_score(y_test, y_pred))
#     return depthlist, depth_r2

In [ ]:
# best_max_depth_calculator()

In [ ]:
# print(depthlist)
# print(depth_r2)

In [53]:
regr = RandomForestRegressor(max_depth=15, random_state=0)
regr.fit(DummiesX, y)
y_pred = regr.predict(X_test)
meansquare = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
r2 = r2_score(y_test, y_pred)
meansquare

74.60472806146221

In [54]:
r2

0.07568568975588619

## RFR oorzaakcode en prioriteit

In [75]:
df_Kpog = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']]
df_Kpog

,#stm_sap_meldnr,stm_hers_tijd,stm_oorz_groep,stm_prioriteit,stm_oorz_code
0,50053211,540.0,ONR-RIB,9.0,133.0
1,50053213,806.0,ONR-DERD,9.0,143.0
2,50053214,1040.0,ONR-DERD,9.0,142.0
3,50053215,1356.0,ONR-DERD,9.0,142.0
4,50053218,690.0,TECHONV,9.0,218.0
...,...,...,...,...,...
461153,99003503,179.0,TECHONV,4.0,221.0
461154,99003504,3.0,TECHONV,5.0,298.0
461155,99003505,24.0,TECHONV,5.0,215.0
461156,99003506,36.0,TECHONV,2.0,218.0


In [76]:
df_Kpog = df_Kpog.drop_duplicates(subset=['#stm_sap_meldnr'])

df_Kpog = df_Kpog.reset_index(drop=True)

df_Kpog = df_Kpog.dropna(subset=['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit']) # , 'stm_oorz_groep'

df_Kpog = df_Kpog.reset_index(drop=True)

In [77]:
# decide what the quantiles are
Q1 = df_Kpog['stm_hers_tijd'].quantile(0.25)
Q3 = df_Kpog['stm_hers_tijd'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

df_Kpog = df_Kpog[(df_Kpog['stm_hers_tijd'] >= 4) & (df_Kpog['stm_hers_tijd'] <= 480)]

df_Kpog = df_Kpog.reset_index(drop=True)

73.0


In [78]:
DummiesX = pd.get_dummies(data=df_Kpog[['stm_oorz_code']]) #, 'stm_oorz_groep']])
DummiesX = pd.concat([DummiesX,df_Kpog['stm_prioriteit']], axis = 1)
DummiesX.head()
# DummiesX = df_Kpog[['stm_prioriteit']]
y = df_Kpog['stm_hers_tijd']

In [79]:
X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)

In [80]:
def best_max_depth_calculator():
    depthlist = []
    depth_r2 = []
    for depth in range(1,15):
        regr = RandomForestRegressor(max_depth=depth, random_state=0)
        regr.fit(DummiesX, y)
        y_pred = regr.predict(X_test)
        depthlist.append(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
        depth_r2.append(r2_score(y_test, y_pred))
    return depthlist, depth_r2

In [93]:
depthlist, depth_r2 = best_max_depth_calculator()

KeyboardInterrupt: 

In [ ]:
depthlist

In [ ]:
depth_r2

In [82]:
regr = RandomForestRegressor(max_depth=15, random_state=0)
regr.fit(DummiesX, y)
y_pred = regr.predict(X_test)
meansquare = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
r2 = r2_score(y_test, y_pred)
meansquare

74.60472806146221

In [83]:
r2

0.07568568975588619

# RFR Functie

In [146]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np

def prep_data (df):
    df_Kpog = df.drop_duplicates(subset=['#stm_sap_meldnr'])
    df_Kpog = df_Kpog.reset_index(drop=True)
    df_Kpog = df_Kpog.dropna(subset=['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit']) # , 'stm_oorz_groep'
    df_Kpog = df_Kpog.reset_index(drop=True)
    df_Kpog = df_Kpog[(df_Kpog['stm_hers_tijd'] >= 4) & (df_Kpog['stm_hers_tijd'] <= 480)]
    df_Kpog = df_Kpog.reset_index(drop=True)
    return df_Kpog

def prep_rfr_model (df):
    DummiesX = pd.get_dummies(data=df[['stm_oorz_code', 'stm_prioriteit']]) #, 'stm_oorz_groep']])
#     DummiesX = pd.concat([DummiesX,df['stm_prioriteit']], axis = 1)
    DummiesX.head()
    # DummiesX = df_Kpog[['stm_prioriteit']]
    y = df['stm_hers_tijd']
    X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)
    return y, X_train, X_test, y_train, y_test
    
def rfr_model (y, X_train, X_test, y_train, y_test):
    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(DummiesX, y)
    y_pred = regr.predict(X_test)
    meansquare = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test, y_pred)
    return r2, meansquare

def rfc_model (y, X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(max_depth=15, random_state=0)
    clf.fit(DummiesX, y)
    y_pred = clf.predict(X_test)
    meansquare = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test, y_pred)
#     print(clf.predict_proba(DummiesX[:5]))
    return r2, meansquare

def decision_tree_regressor_predict_proba(X_train, y_train, X_test):
    """Trains DecisionTreeRegressor model and predicts probabilities of each y.

    Args:
        X_train: Training features.
        y_train: Training labels.
        X_test: New data to predict on.
        **kwargs: Other arguments passed to DecisionTreeRegressor.

    Returns:
        DataFrame with columns for record_id (row of X_test), y 
        (predicted value), and prob (of that y value).
        The sum of prob equals 1 for each record_id.
    """
    # Train model.
    m = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
    # Get y values corresponding to each node.
    node_ys = pd.DataFrame({'node_id': m.apply(X_train), 'y': y_train})
    # Calculate probability as 1 / number of y values per node.
    node_ys['prob'] = 1 / node_ys.groupby(node_ys.node_id).transform('count')
    # Aggregate per node-y, in case of multiple training records with the same y.
    node_ys_dedup = node_ys.groupby(['node_id', 'y']).prob.sum().to_frame()\
        .reset_index()
    # Extract predicted leaf node for each new observation.
    leaf = pd.DataFrame(m.decision_path(X_test).toarray()).apply(
        lambda x:x.to_numpy().nonzero()[0].max(), axis=1).to_frame(
            name='node_id')
    leaf['record_id'] = leaf.index
    # Merge with y values and drop node_id.
    return leaf.merge(node_ys_dedup, on='node_id').drop('node_id', axis=1).sort_values(['record_id', 'y'])

In [147]:
df_Kpog = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']]

In [148]:
df_Kpog = prep_data (df_Kpog)
y, X_train, X_test, y_train, y_test = prep_rfr_model (df_Kpog)
# print(rfc_model(y, X_train, X_test, y_train, y_test))
# print(rfr_model(y, X_train, X_test, y_train, y_test))
# decision_tree_regressor_predict_proba(X_train, y_train, X_test)

,record_id,y,prob
0,0,4.0,0.007833
1,0,5.0,0.011333
2,0,6.0,0.006333
3,0,7.0,0.006167
4,0,8.0,0.006833
...,...,...,...
5465236,86119,474.0,0.000197
5465237,86119,476.0,0.000066
5465238,86119,478.0,0.000066
5465239,86119,479.0,0.000262
